The Lorenz system is a system of three ODEs having chaotic solutions (butterfly effect).

$$\begin{split} \dot{x} & = \sigma (y - x) \\
\dot{y} & = x (\rho - z) - y \\
\dot{z} & = xy - \beta z
\end{split}$$

Reference: https://en.wikipedia.org/wiki/Lorenz_system

In [12]:
from lmmNet import lmmNet
import numpy as np
from scipy.integrate import odeint

In [8]:
def f(x,t):
    """
    The system of ODEs for the nonlinear Lorenz system with default parameters
    
    Arguments:
    x --  a list with three elements corresponding to the three variables
    t -- time
    
    Return:
    A numpy array containing the derivatives
    """
    
    sigma = 10.0
    beta = 8.0/3.0
    rho = 28.0

    r1 = sigma * (x[1] - x[0])
    r2 = x[0] * (rho - x[2]) - x[1]
    r3 = x[0] * x[1] - beta * x[2]

    return np.array([r1, r2, r3])

In [9]:
# initial condition
x0 = np.array([-8.0, 7.0, 27])

t0 = 0 # start time
T = 25 # end time
h = 0.01 # step size

# generate data
time_points = np.arange(t0, T, h)
lorenz_data = odeint(f, x0, time_points)

In [11]:
# add Gaussian noise scaled by standard deviation, for every one of the three dimensions
noise_strength = 0.00
lorenz_data += noise_strength * lorenz_data.std(0) * np.random.randn(lorenz_data.shape[0], lorenz_data.shape[1])

lorenz_data = np.reshape(lorenz_data, (1,lorenz_data.shape[0], lorenz_data.shape[1]))

In [14]:
hidden_layer_units = 256 # number of units for the hidden layer
M = 1 # number of steps
scheme = 'AM' # LMM scheme
model = lmmNet(h, lorenz_data, M, scheme, hidden_layer_units)

N_Iter = 10000
model.train(N_Iter)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch: 0, Loss: 1.245e+00, Time: 0.18
Epoch: 100, Loss: 1.029e+00, Time: 7.31
Epoch: 200, Loss: 8.478e-01, Time: 14.45
Epoch: 300, Loss: 7.074e-01, Time: 22.25
Epoch: 400, Loss: 6.008e-01, Time: 30.91
Epoch: 500, Loss: 5.131e-01, Time: 38.26
Epoch: 600, Loss: 4.428e-01, Time: 45.21
Epoch: 700, Loss: 3.860e-01, Time: 54.18
Epoch: 800, Loss: 3.384e-01, Time: 63.99
Epoch: 900, Loss: 2.983e-01, Time: 71.88
Epoch: 1000, Loss: 2.640e-01, Time: 80.25
Epoch: 1100, Loss: 2.346e-01, Time: 88.76
Epoch: 1200, Loss:

In [16]:
def ml_f(x, model):
    """
    Define the derivatives learned by ML
    I think this is the best implementation, more robust than flatten()
    """
    return np.ravel(model.predict(x.reshape(1,-1)))